In [1]:
import gc
import json
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm

from utils import loadpkl, removeCorrelatedVariables, removeMissingVariables

%matplotlib inline

In [17]:
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

# set card_id as index
df.set_index('card_id', inplace=True)
df.head()

  0%|          | 0/1091 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 1091/1091 [00:01<00:00, 626.79it/s]


,AOV,AOV_ratio,CLV,CLV_month_diff,CLV_month_diff_ratio,CLV_ratio,Mothers_Day_2018,amount_month_ratio_max,amount_month_ratio_mean,amount_month_ratio_min,...,weekofyear_9,year_0,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,182.318395,1.661881,4.690294e+06,1.705895e+08,-0.008053,0.013005,49.492977,NaN,NaN,-350.00,...,0,0,0,0,0,0,0,0,1,0
C_ID_3d0044924f,101.445060,0.159410,1.071890e+07,1.005798e+07,0.000050,0.000047,56.833333,NaN,NaN,7.99,...,0,0,0,0,0,0,0,0,1,0
C_ID_d639edf6cd,76.651163,0.679063,8.444000e+04,NaN,NaN,0.000367,15.720930,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
C_ID_186d6a6901,118.311688,0.856153,3.805900e+05,NaN,NaN,0.007076,55.623377,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
C_ID_cdbd2c0db2,249.208020,1.067407,2.299008e+06,-9.753155e+07,0.031752,0.078204,61.856099,NaN,NaN,-156.09,...,0,0,0,0,0,0,0,0,1,0


In [21]:
configs = json.load(open('../configs/207_lgbm_best.json'))
df = df[configs['features']]
df.head()

,target,outliers,CLV_ratio,amount_month_ratio_max,amount_month_ratio_mean,amount_month_ratio_min,card_id_cnt_ratio,card_id_cnt_total,card_id_total,category_1_mean,...,new_weekofyear_nunique,price_max,price_mean,price_total,purchase_amount_max,purchase_amount_mean,purchase_amount_min,purchase_amount_ratio,purchase_amount_total,quarter
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,0.0,0.013005,NaN,NaN,-350.00,0.088462,283.0,283.0,0.000000,...,7.0,1329.000,11856.00000,5108.00000,1329.0,182.3750,20.000000,0.146973,20432.0,2.0
C_ID_3d0044924f,0.392913,0.0,0.000047,NaN,NaN,7.99,0.017143,356.0,356.0,0.088571,...,4.0,96.250,39.53125,55.71875,1059.0,101.4375,7.988281,0.002733,30704.0,1.0
C_ID_d639edf6cd,0.688056,0.0,0.000367,NaN,NaN,NaN,0.023256,44.0,44.0,0.000000,...,1.0,NaN,NaN,NaN,431.0,76.6250,42.156250,0.015793,1994.0,3.0
C_ID_186d6a6901,0.142495,0.0,0.007076,NaN,NaN,NaN,0.090909,84.0,84.0,0.298701,...,5.0,287.250,55.12500,57.46875,1149.0,118.3125,12.500000,0.077820,5288.0,3.0
C_ID_cdbd2c0db2,-0.159749,0.0,0.078204,NaN,NaN,-156.09,0.270677,169.0,169.0,0.168338,...,8.0,130.375,104.00000,94.81250,1826.0,249.2500,5.500000,0.288818,20672.0,4.0


In [12]:
corr = df.corr()
corr.to_csv('../output/corr.csv')
corr

,feature_1,feature_2,feature_3,outliers,target,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,...,hist_CLV,new_pred_CLV,hist_pred_CLV,new_CLV_month_diff_ratio,hist_CLV_month_diff_ratio,CLV_ratio,AOV_ratio,pred_CLV_ratio,CLV_month_diff_ratio,observation_date_ratio
feature_1,1.000000,-0.160334,0.451606,0.012110,-0.013809,-0.013137,0.103627,0.451741,-0.058931,0.297670,...,-0.001708,-0.046730,0.170198,0.071792,0.013337,-0.008906,-0.030447,-0.002070,-0.008613,0.005549
feature_2,-0.160334,1.000000,0.068321,0.018778,-0.015389,0.024569,0.098377,0.073599,0.235076,0.232950,...,-0.056704,0.030003,-0.001487,0.003744,-0.015925,-0.008423,-0.000120,-0.004340,-0.011413,-0.005473
feature_3,0.451606,0.068321,1.000000,0.005457,-0.008125,-0.018313,0.195722,0.393836,0.174719,0.670172,...,-0.055178,-0.105429,0.010252,0.060519,-0.005671,-0.008051,-0.001647,-0.000438,-0.008225,0.008734
outliers,0.012110,0.018778,0.005457,1.000000,-0.896187,-0.034347,0.053738,0.051106,0.041644,0.036989,...,-0.008064,-0.011633,0.009532,0.017514,-0.007912,-0.003340,0.003197,-0.001470,-0.003920,0.041955
target,-0.013809,-0.015389,-0.008125,-0.896187,1.000000,0.032560,-0.050453,-0.049299,-0.039142,-0.035481,...,-0.004075,-0.045824,-0.009972,-0.012225,0.007405,-0.007692,-0.086001,-0.005233,-0.004712,-0.046615
quarter,-0.013137,0.024569,-0.018313,-0.034347,0.032560,1.000000,-0.228275,-0.176459,-0.164910,-0.119154,...,-0.064703,0.025878,-0.088000,-0.058270,-0.040707,0.016470,-0.008843,0.009240,0.006188,-0.018478
elapsed_time,0.103627,0.098377,0.195722,0.053738,-0.050453,-0.228275,1.000000,0.875903,0.833557,0.770325,...,0.078418,-0.063742,0.139303,0.080527,0.035412,-0.013479,0.014737,-0.011378,-0.004400,0.001309
days_feature1,0.451741,0.073599,0.393836,0.051106,-0.049299,-0.176459,0.875903,1.000000,0.636853,0.837205,...,0.085463,-0.030726,0.229964,0.074854,0.035995,-0.014101,0.004212,-0.010398,-0.007705,0.002820
days_feature2,-0.058931,0.235076,0.174719,0.041644,-0.039142,-0.164910,0.833557,0.636853,1.000000,0.662535,...,0.029162,-0.087926,0.010864,0.069084,0.015192,-0.007339,0.012538,-0.008521,-0.001897,-0.002204
days_feature3,0.297670,0.232950,0.670172,0.036989,-0.035481,-0.119154,0.770325,0.837205,0.662535,1.000000,...,0.003191,-0.079984,0.090556,0.065519,0.010137,-0.014508,0.007353,-0.007746,-0.009065,0.009510


In [22]:
# split test & train
train_df = df[df['target'].notnull()]
test_df = df[df['target'].isnull()]

In [23]:
# Kolmogorov-Smirnov statistic on 2 samples

from scipy.stats import ks_2samp
list_p_value =[]

for c in tqdm(train_df.columns.tolist()):
    list_p_value.append(ks_2samp(test_df[c] , train_df[c])[1])

Se = pd.Series(list_p_value, index = train_df.columns.tolist()).sort_values() 
list_discarded = list(Se[Se < .1].index)

100%|██████████| 195/195 [00:07<00:00, 27.43it/s]


In [24]:
list_discarded

['target',
 'outliers',
 'purchase_amount_max',
 'purchase_amount_total',
 'new_category_2_mean_mean',
 'hist_purchase_amount_sum',
 'new_category_2_mean',
 'new_purchase_amount_mean',
 'purchase_amount_mean',
 'hist_purchase_amount_max',
 'new_hour_mean',
 'new_hour_min',
 'hist_category_2_mean_mean',
 'hist_category_3_mean',
 'installments_mean',
 'hist_installments_mean']